In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from transformers import MBartForConditionalGeneration, AutoTokenizer,MBart50Tokenizer,MBart50TokenizerFast,AutoModelForSeq2SeqLM

In [3]:
# ✅ Your Custom Summary Function
def generate_summary(text, model, tokenizer, device):
    model.to(device)

    # Tokenize input text
    inputs = tokenizer(text, max_length=512, truncation=True, return_tensors="pt", padding="max_length").to(device)

    # Generate summary
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [4]:
model_path = "/content/drive/My Drive/Thesis_Dataset/fine_tuned_mt5"

# Load tokenizer and model correctly
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load transcribed text from Excel
input_excel = "/content/drive/MyDrive/Thesis_Transcription/Text_Output/chunk_transcripts.xlsx"
df = pd.read_excel(input_excel)

# Apply your summary function to each row
print("📄 Generating summaries for all transcribed chunks...")
df['Summary'] = df['Text'].progress_apply(lambda x: generate_summary(str(x), model, tokenizer, device))

# Save to new Excel
output_excel_path = "/content/drive/MyDrive/Thesis_Transcription/Text_Output/text_with_summaries_mt5.xlsx"
df.to_excel(output_excel_path, index=False)

print(f"✅ Excel with summaries saved: {output_excel_path}")

# Save all summaries as one merged text
merged_summary_text = "\n".join(df['Summary'].dropna().astype(str).tolist())
summary_txt_path = "/content/drive/MyDrive/Thesis_Transcription/Text_Output/merged_summary_mt5_without_regex.txt"
with open(summary_txt_path, "w", encoding="utf-8") as f:
    f.write(merged_summary_text)



HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/My Drive/Thesis_Dataset/fine_tuned_mt5'. Use `repo_type` argument if needed.